# Kernel helpers

Here is a complete reference of Michelson kernel helpers. 

> Note: These instructions are not Michelson primitives and thus cannot be used outside of the Jupyter environment.

### DUMP
Returns the whole stack with values, types, and annotations if any.

In [1]:
PUSH @var nat 42 ;
DUMP

value,type,name
42,nat,@var


### DUMP n
Returns top `n` items from the stack.

In [2]:
PUSH string "1" ;
PUSH string "2" ;
PUSH string "3" ;
DUMP 2

PUSH: push 1;
PUSH: push 2;
PUSH: push 3;

value,type
"""3""",string
"""2""",string


### DROP_ALL
Clears the stack.

In [3]:
DROP_ALL ;
DUMP

stack is empty

### EXPAND { code }
Expands Michelson macros.

In [4]:
EXPAND { PAPAIIR }

{ { DIP { PAIR } ; PAIR } }

### PATCH prim value?
Sets value for one of the context-dependent Michelson instructions: `AMOUNT`, `BALANCE`, `NOW`, `SOURCE`, `SENDER`, `CHAIN_ID`.  
You can leave second parameter empty, `0` will be used in that case.

In [5]:
PATCH AMOUNT 100500 ;
AMOUNT

PATCH: set AMOUNT=100500;
AMOUNT: push 100500;

value,type,name
100500,mutez,@amount


### DEBUG bool
Enables or disables verbose output.

In [6]:
DEBUG False ;
PUSH nat 42

value,type
42,nat


In [7]:
DEBUG True

### BIG_MAP_DIFF
Takes the top of the stack, searches for temporary big_map instances in that element, and displays what the `big_map_diff` would be like if it was a contract execution ending.

In [8]:
EMPTY_BIG_MAP string string ;
PUSH string "bar" ;
SOME ;
PUSH string "foo" ;
UPDATE ;
BIG_MAP_DIFF

EMPTY_BIG_MAP: push -1;
PUSH: push bar;
SOME: pop bar; push ('bar',);
PUSH: push foo;
UPDATE: pop foo, ('bar',), -1; push -1;
BIG_MAP_DIFF:

big_map,action,key,value
0,alloc,string,string
0,update,"""foo""","""bar"""


### BEGIN %entrypoint (param_expr) (storage_expr)
Simulates the contract execution beginning. Requires `parameter` and `storage` sections to be initialized. Also, **clears the stack**.  
The `%entrypoint` argument can be omitted, `%default` will be used in that case.  
This helper also allocates temporary `big_map` instances if there are any in parameters or storage types.

In [9]:
parameter unit ;
storage unit ;
BEGIN Unit Unit ;

parameter unit;
storage unit;
BEGIN: use %default; drop all; push (Unit, Unit);

value,type
Pair Unit Unit,pair unit unit


### COMMIT
Simulates the contract execution ending. Requires a `Pair` of operation list and valid storage on top of the stack.  
Returns a `list` of internal operations, new storage, and `big_map_diff`.

In [10]:
DROP ;
UNIT ;
NIL operation ;
PAIR ;
COMMIT  ## in our case it will be just new storage

DROP: pop (Unit, Unit);
UNIT: push Unit;
NIL: push [];
PAIR: pop [], Unit; push ([], Unit);
COMMIT:

value,type
Unit,unit


### RESET network?
**Clears the stack**, deletes all big_map instances, initializes `CHAIN_ID` variable, and remembers the chosen network.  
Can be used to set real network context in order to access blockchain data and typecheck values of `contract` type.  
If _network_ is not specified, `CHAIN_ID` will we unset.

In [11]:
PUSH address "KT18jLSuXycuWi9pL7ByRjqkrWpPF1S6maHY" ;
CONTRACT unit ;  ## no typechecking
ASSERT_SOME ;

PUSH: push KT18jLSuXycuWi9pL7ByRjqkrWpPF1S6maHY;
CONTRACT: pop KT18jLSuXycuWi9pL7ByRjqkrWpPF1S6maHY; skip check; push ('KT18jLSuXycuWi9pL7ByRjqkrWpPF1S6maHY%default',);
IF_NONE: pop ('KT18jLSuXycuWi9pL7ByRjqkrWpPF1S6maHY%default',); push KT18jLSuXycuWi9pL7ByRjqkrWpPF1S6maHY%default;
  RENAME: pop KT18jLSuXycuWi9pL7ByRjqkrWpPF1S6maHY%default; push KT18jLSuXycuWi9pL7ByRjqkrWpPF1S6maHY%default;

value,type
"""KT18jLSuXycuWi9pL7ByRjqkrWpPF1S6maHY%default""",contract unit


In [12]:
RESET "mainnet" ;
CHAIN_ID ;
PUSH address "KT18jLSuXycuWi9pL7ByRjqkrWpPF1S6maHY" ;
CONTRACT unit ;
ASSERT_SOME ;

RESET: set NETWORK='mainnet'; set CHAIN_ID=NetXdQprcVkpaWU; drop all;
CHAIN_ID: push NetXdQprcVkpaWU;
PUSH: push KT18jLSuXycuWi9pL7ByRjqkrWpPF1S6maHY;
CONTRACT: pop KT18jLSuXycuWi9pL7ByRjqkrWpPF1S6maHY; not found; push None;
IF_NONE: pop None;
  UNIT: push Unit;
  FAILWITH: pop Unit;

MichelsonRuntimeError: Unit
at IF_NONE -> FAILWITH

In [13]:
RESET

### RUN %entrypoint (param_expr) (storage_expr)
Requires `code` section to be initialized. Internally calls `BEGIN`, then executes `code`, and finishes with `COMMIT`.

In [14]:
parameter unit ;
storage unit ;
code { CDR ; NIL operation ; PAIR }

parameter unit;
storage unit;
code { CDR ; NIL operation ; PAIR };

In [15]:
RUN Unit Unit

RUN: use %default; drop all; push (Unit, Unit);
  CDR: pop (Unit, Unit); push Unit;
  NIL: push [];
  PAIR: pop [], Unit; push ([], Unit);

value,type
Unit,unit
